In [ ]:
# Import python packages
import os
import sys
import laspy

import geopandas as gpd
import pandas as pd 

from sqlalchemy import create_engine

# Add parent folder to path, so that notebook can find .py scripts
DIR_BASE = os.path.abspath(os.path.join('..'))
if DIR_BASE not in sys.path:
    sys.path.append(DIR_BASE)

# Import functions from own .py scripts
import config
from pointcloud_functions import \
    add_geoindex_to_databases, \
    load_laz_pointcloud_into_database, \
    add_floor_points_to_points_in_gdf, \
    crop_and_fetch_pointclouds_per_building, \
    save_lidar_numpy_list

from utils.utils import convert_multipoint_to_numpy, check_directory_paths, file_name_from_polygon_list
from utils.visualization import visualize_single_3d_point_cloud
from utils.aerial_image import get_aerial_image_lat_lon

# Configurations
Here, we define filepaths and parameters for pointcloud creation  

In [ ]:
# Define pointcloud parameters
AREA_OF_INTEREST_CODE = 'E06000014' # UK local authority boundary code to specify area of interest (AOI)
BUILDING_BUFFER_METERS = 0.5 # buffer around building footprint in meters
MAX_NUMBER_OF_FOOTPRINTS = None  # define how many footprints should be created. Use "None" to use all footprints in AOI
POINT_COUNT_THRESHOLD = 100  # define minimum points in pointcloud, smaller pointclouds are dismissed
NUMBER_EXAMPLE_VISUALIZATIONS = 10  # define how many example 3D plots should be created

# Define project base directory and paths
# DIR_BASE = os.getcwd() # in jupyter, use a different approach (above) to determine project DIR
DIR_ASSETS = os.path.join(DIR_BASE, 'assets')
DIR_OUTPUTS = os.path.join(DIR_BASE, 'outputs')

DIR_LAZ_FILES = os.path.join(DIR_ASSETS, "uk_lidar_data")
DIR_EPC = os.path.join(DIR_ASSETS, "epc")
DIR_VISUALIZATION = os.path.join(DIR_ASSETS, "example_pointclouds")
DIR_AERIAL_IMAGES = os.path.join(DIR_ASSETS, "aerial_image_examples")

# Create a new output folder for the defined area of interest
DIR_AOI_OUTPUT = os.path.join(DIR_OUTPUTS, AREA_OF_INTEREST_CODE)
if os.path.isdir(DIR_AOI_OUTPUT):
    print('output for this area of interest already exists. delete or choose other area code')
else:
    os.mkdir(DIR_AOI_OUTPUT)
DIR_NPY = os.path.join(DIR_AOI_OUTPUT, 'npy_raw')
if not os.path.isdir(DIR_NPY): os.mkdir(DIR_NPY)


# Check that all required directories exist
check_directory_paths([DIR_ASSETS, DIR_OUTPUTS, DIR_LAZ_FILES, DIR_VISUALIZATION, DIR_AERIAL_IMAGES, DIR_AOI_OUTPUT, DIR_NPY])

DB_TABLE_NAME_LIDAR = 'uk_lidar_data'
DB_TABLE_NAME_FOOTPRINTS = 'footprints_verisk'
DB_TABLE_NAME_UPRN = 'uprn'
DB_TABLE_NAME_AREA_OF_INTEREST = 'local_authority_boundaries'


# Main
The following code elements 
- create a database connection and test if it was successful
- load pointcloud LAZ files into the database
- make sure the tables in the database contain a georeference, to ensure DB query is as fast as possible
- generate building pointclouds based by cropping points within the building's footprint
- add a building's floorpoints artificially, because those are missing in airborne LiDAR data
- save the pointclouds as .npy files and save required data (except LiDAR tiles) as .geojson
- visualize a subset of the pointclouds and their arial images for evaluation

In [ ]:
# Intialize connection to database
db_connection_url = config.DATABASE_URL
engine = create_engine(db_connection_url, echo=False)

In [ ]:
# Test connection to database
# with engine.connect() as con:
#     res = con.execute('SELECT * FROM footprints_verisk LIMIT 1')
# print(res.all())

In [ ]:
# Load footprint geojsons into database (only required if they haven't already been uploaded already)
# STANDARD_CRS = 27700
# DIR_BUILDING_FOOTPRINTS = os.path.join(DIR_ASSETS, "aoi")
# gdf_footprints = load_geojson_footprints_into_database(
#     DIR_BUILDING_FOOTPRINTS, DB_TABLE_NAME_FOOTPRINTS, engine, STANDARD_CRS
# )

In [ ]:
# Load point cloud data into database
# Unpacks LAZ-files and inserts all newly unpacked LAS-files into the database
# Existing LAS-files in directory are considered to be in the database already
load_laz_pointcloud_into_database(DIR_LAZ_FILES, DB_TABLE_NAME_LIDAR)

In [ ]:
# Load EPC data into database
file_path = os.path.join(DIR_EPC, AREA_OF_INTEREST_CODE + '.csv')
df_epc = pd.read_csv(file_path)
with engine.connect() as con:
    df_epc.to_sql('epc', con=con, if_exists='replace', index=False)

In [ ]:
# Add geoindex to footprint and lidar tables and vacuum table 
db_table_names = [DB_TABLE_NAME_LIDAR, DB_TABLE_NAME_FOOTPRINTS, DB_TABLE_NAME_UPRN, DB_TABLE_NAME_AREA_OF_INTEREST]
db_is_lidar = [1, 0, 0, 0]
add_geoindex_to_databases(config.DATABASE_URL, db_table_names, db_is_lidar)

todo: adapt SQL query to use only footprints that contain UPRN

In [ ]:
# Adapt NUMBER_OF_FOOTPRINTS to use all footprints if None
if MAX_NUMBER_OF_FOOTPRINTS == None:
    MAX_NUMBER_OF_FOOTPRINTS = 1000000000  # 1 billion, which is more than UKs building stock

# Fetch cropped point clouds from database
gdf = crop_and_fetch_pointclouds_per_building(
    AREA_OF_INTEREST_CODE, BUILDING_BUFFER_METERS, MAX_NUMBER_OF_FOOTPRINTS, POINT_COUNT_THRESHOLD, engine)


In [ ]:
# Test print some gdf entry examples (york)
#print(gdf[gdf.id_fp==9664325]) # no uprn
#print(gdf[gdf.id_fp==9600120]) # no pc
#print(gdf[gdf.id_fp==9681016]) # all 3 geoms exist

In [ ]:
# Add floor points to building pointcloud
gdf_pc = gdf[gdf.geom!=None]
gdf_pc = add_floor_points_to_points_in_gdf(gdf_pc)

In [ ]:
# Save raw pointcloud without threshhold or scaling
lidar_numpy_list = list(gdf_pc.geom.apply(convert_multipoint_to_numpy))
# Save building point clouds as npy
save_lidar_numpy_list(lidar_numpy_list, gdf_pc, DIR_NPY)

# Save raw information of footprints, epc label, uprn, file mapping

In [ ]:
# footprints
gdf_footprints = gpd.GeoDataFrame({"if_fp": gdf.id_fp, "geometry": gdf.geom_fp})
gdf_footprints = gdf_footprints.drop_duplicates()
save_path = os.path.join(DIR_AOI_OUTPUT, str('footprints_' + AREA_OF_INTEREST_CODE + ".json"))
gdf_footprints.to_file(save_path, driver="GeoJSON")  

In [ ]:
# uprn
gdf_uprn = gpd.GeoDataFrame({"uprn": gdf.uprn, "geometry": gdf.geom_uprn})
gdf_uprn = gdf_uprn.drop_duplicates()
save_path = os.path.join(DIR_AOI_OUTPUT, str('uprn_' + AREA_OF_INTEREST_CODE + ".json"))
gdf_uprn.to_file(save_path, driver="GeoJSON")  

In [ ]:
# epc label
gdf_epc = pd.DataFrame(
    {"id_epc_lmk_key": gdf.id_epc_lmk_key, 
     "rating": gdf.energy_rating,
     "efficiency": gdf.energy_efficiency}
)
gdf_epc = gdf_epc.drop_duplicates()
save_path = os.path.join(DIR_AOI_OUTPUT, str('epc_' + AREA_OF_INTEREST_CODE + ".json"))
gdf_epc.to_json(save_path, orient='records') 

In [ ]:
# label - filename mapping
file_names = file_name_from_polygon_list(list(gdf.geom_fp), file_extension='.npy')
gdf_mapping = pd.DataFrame(
    {"id_fp": gdf.id_fp,
     "uprn": gdf.uprn,
     "id_id_epc_lmk_key": gdf.id_epc_lmk_key,
     "id_query": gdf.id_query,
     "epc_rating": gdf.energy_rating,
     "epc_efficiency": gdf.energy_efficiency,
     "file_name": file_names
    } 
)
save_path = os.path.join(DIR_AOI_OUTPUT, str('label_filename_mapping_' + AREA_OF_INTEREST_CODE + ".json"))
gdf_mapping.to_json(save_path, orient='index') 

# Visualization for evaliation of results

In [ ]:
# Visualize example building pointcloud data 
for i, lidar_pc in enumerate(lidar_numpy_list):
    if i <= NUMBER_EXAMPLE_VISUALIZATIONS:
        pce_file_name = file_name_from_polygon_list(list(gdf.iloc[i].geom_fp), file_extension=".html")
        save_path = os.path.join(DIR_VISUALIZATION, pce_file_name)
        visualize_single_3d_point_cloud(
            lidar_pc,
            title=str(i),
            save_path=save_path,
            show=False
        )

In [ ]:
# Download aerial image for the building examples
gdf_lat_lon = gdf.to_crs(4326)

for i, building in enumerate(gdf_lat_lon.iloc):
    img_filename = file_name_from_polygon_list(list(gdf.iloc[i].geom_fp), file_extension=".png")
    if i <= NUMBER_EXAMPLE_VISUALIZATIONS:
        cp = building.geom.centroid
        get_aerial_image_lat_lon(
            latitude=cp.y,
            longitude=cp.x,
            image_name=img_filename,
            horizontal_px=512,
            vertical_px=512,
            scale=1,
            zoom=21,
            save_directory=DIR_AERIAL_IMAGES
        )

# ARCHIVE

### Raw code of LAZ to DB for debugging

In [ ]:
archive: are you sure you want to run this?
    
# get files in directory
files_uk_lidar = os.listdir(DIR_LAZ_FILES)

# check which laz files have not yet been unpacked
laz_files = [file for file in files_uk_lidar if file[-4:] == ".laz"]
las_files = [file for file in files_uk_lidar if file[-4:] == ".las"]
unpacked_files = [laz_file for laz_file in laz_files if not laz_file[:-4] + '.las' in las_files]
#unpacked_files = laz_files
import pdal
import json

print('Loading pointcloud data from las to database. This process can take several minutes')
# unzip LAZ files, if corresponding LAS file does not exist
for i, unpacked_file in enumerate(unpacked_files):
    print('unpacking laz file %s of %s: %s' % (str(i + 1), str(len(unpacked_files)), unpacked_file))
    # unzip laz to las
    in_laz = os.path.join(DIR_LAZ_FILES, unpacked_file)
    #out_las = os.path.join(DIR_LAZ_FILES, unpacked_file)
    out_las = os.path.join(DIR_LAZ_FILES, unpacked_file[:-4] + '.las')
    las = laspy.read(in_laz)
    las = laspy.convert(las)
    print('converted laz to las')
    las.write(out_las)
    # load las files into database
    print('loading files into database')
    las_to_db_pipeline = {
        "pipeline": [
            {
                "type": "readers.las",
                "filename": out_las,
                "spatialreference": "EPSG:27700"
            },
            {
                "type": "filters.chipper",
                "capacity": 800
            },
            {
                "type": "writers.pgpointcloud",
                "connection": "host='%s' dbname='%s' user='%s' password='%s' port='%s'" %
                              (config.POSTGRES_HOST, config.POSTGRES_DATABASE,
                               config.POSTGRES_USER, config.POSTGRES_PASSWORD,
                               config.POSTGRES_PORT),
                "schema": "public",
                "table": DB_TABLE_NAME_LIDAR,
                "compression": "dimensional",
                "srid": "27700",
                "overwrite": "false"
            }
        ]
    }

    print('loading laz file %s of %s into database' % (str(i + 1), str(len(unpacked_files))))
    pipeline = pdal.Pipeline(json.dumps(las_to_db_pipeline))
    pipeline.execute()

print('Loading data into database finished')
